In [ ]:
from absl.testing import absltest
from absl.testing import parameterized
from brax import envs
from brax.training import apg

a, b, metrics = apg.train(
    environment_fn=envs.create_fn('acrobot'),
    episode_length=500,
    action_repeat=1,
    num_envs=16,
    learning_rate=1e-3,
    normalize_observations=True,
    log_frequency=200,
    truncation_length=25,
)

In [ ]:
metrics